In [61]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url="https://www.leboncoin.fr/voitures/offres/ile_de_france/?th=1&q=renault%20zoe"

def getSoupFromURL(url, method='get', data={}):

    if method == 'get':
        res = requests.get(url)
    elif method == 'post':
        res = requests.post(url, data=data)
    else:
        return None

    if res.status_code == 200:
        soup = BeautifulSoup(res.text, 'html.parser')
        return soup
    else:
        return None
    

def truc(url):
    list = []
    soup = getSoupFromURL(url)
    if soup:
        #a = soup.find_all(class_="item_infos")
        a = soup.find_all(class_="tabsContent")
        for  a0 in a:
            a1 = a0.find_all("li")
            
            for a2 in a1:
                urlItem = "http:" + a2.find ("a")["href"]
                #print (a2)
                if urlItem != None:
                    soupItem = getSoupFromURL(urlItem)
                    if soupItem:
                        #inspecter chaque élément
                        b = soupItem.find_all(class_="line")
                        for b0 in b:
                            #print (b0)
                            b1 = b0.find_all(class_="property")
                            b1Prime = b0.find_all(class_="value")
                            for b2  in b1:
                                print (b2.text)
                                
                            for  b2Prime in b1Prime:
                                
                                print (b2Prime.text)
                            
                
                print(a2.find(class_="item_title").text)
                if a2.find(class_="ispro")!= None:
                    print(a2.find(class_="ispro").text)
                else:
                    print ('(NonPro)')
                print(a2.find(class_="item_price").text)
            
    else:
        0
    return list

truc(url)


Prix

                    
                            9 490 €
                        
                    
                    
                

Ville


Livry-Gargan 93190
                
Marque
Renault
Modèle
Zoe
Année-modèle

                    
                        2013
                            
                        
                    
                
Kilométrage
17 900 KM
Carburant
Electrique
Boîte de vitesse
Automatique
Description :
SUPERBE ZOE ZEN CHARGE RAPIDE DE 02/2013.VEHICULE EN PARFAIT ETAT DE MARCHE ET DE PRESENTATION INTERIEUR COMME EXTERIEUR.VEHICULE EQUIPE DE: (IDEM INTENS)-CLIM. AUTO.-R LINK GPS COULEUR, AUDIO BLUETOOTH, TELEPHONE BLUETOOTH+USB ETC...-DIFUSEUR DE PARFUM.-4 VITRES ELECTRIQUES.-REGULATEUR DE VITESSE.-RADAR DE RECUL.-JANTES ALU 16'-ACCOUDOIR CENTRAL.-CARTE MAIN LIBRES DEMARRAGE ET OUVERTURE/FERMETURE SANS CLEFS.-RETROVISEURS ELECTRIQUES.-ALLUMAGE AUTO DES FEUX.-ESSUI GLACES AUTO.-ABS/ESPETC ETC...VEHICULE TOUJOURS REVISE CHEZ RENAULT, LE

Prix

                    
                            6 790 €
                        
                    
                    
                

Ville


Fontenay-le-Vicomte 91540
                
Marque
Renault
Modèle
Zoe
Année-modèle

                    
                        2013
                            
                        
                    
                
Kilométrage
70 896 KM
Carburant
Electrique
Boîte de vitesse
Automatique
Référence
28383
Description :
Renault Zoe LIFE CHARGE RAPIDE berline, blanc, 1 cv, 5 portes, mise en circulation le 27/03/2013, première main, garantie 3 mois.5 placesOPTIONS ET EQUIPEMENTS :Sécurité :- ABS- ESP- aide au freinage d'urgence- répartiteur électronique de freinageAutres équipements et informations :kilométrage garanti, puissance réelle : 88 ch, 3CEINTSECU, ABS, AFU, AIDEPENTE, AIRBAG, AIRBAG1, AIRBAG4, ANTIDEMEL, AP, BACPTAV, BANQMONOBL, BANQRAB, BECQAR, BGANTFERM, BOUCLIERCO, BQUETTE2PL, CDESVOCALE, CDESVOLANT, CEINTAVHT, CHAUF

Prix

                    
                            8 000 €
                        
                    
                    
                

Ville


Aulnay-sur-Mauldre 78126
                
Marque
Renault
Modèle
Zoe
Année-modèle

                    
                        2013
                            
                        
                    
                
Kilométrage
55 300 KM
Carburant
Electrique
Boîte de vitesse
Manuelle
Description :
RENAULT ZOE ZEN   BLANC GLACIERPEINTURE IMPECCABLE1ERE MISE EN CIRCULATION 28/03/2013VOITURE VISIBLE AU RENAULT EPAIGNES 27260

                            	Renault zoe
                                
                            	
							
(NonPro)

                                            8 000 €
                                            
                                        
Prix

                    
                            9 900 €
                        
                    
                    
                

V

Prix

                    
                            10 700 €
                        
                    
                    
                

Ville


Paris 75008
                
Marque
Renault
Modèle
Zoe
Année-modèle

                    
                        2015
                            
                        
                    
                
Kilométrage
40 095 KM
Carburant
Electrique
Boîte de vitesse
Automatique
Référence
a179000752
Description :
Renault Zoe INTENS berline, noir, 1 cv, 5 portes, mise en circulation le 24/04/2015, garantie 6 mois.OPTIONS ET EQUIPEMENTS :Extérieur :- allumage automatique des feux- rétroviseurs électriques- caméra de recul- jantes aluminiumIntérieur et confort :- climatisation automatiqueSécurité :- rétroviseurs dégivrants- radar d'aide au stationnement- ABS- aide au freinage d'urgence- fixation ISOFIX- régulateur de vitesse- limiteur de vitesseAutres équipements et informations :vitres avant éléctriques, feux de circulation diurne

Prix

                    
                            17 790 €
                        
                    
                    
                

Ville


Brie-Comte-Robert 77170
                
Marque
Renault
Modèle
Zoe
Année-modèle

                    
                        2017
                            
                        
                    
                
Kilométrage
1 KM
Carburant
Electrique
Boîte de vitesse
Automatique
Référence
vo221069
Description :
Renault Zoe Intens Gamme 2017 berline, gris titanium, 1 cv, 5 portes, mise en circulation le 28/09/2017, garantie 24 mois.OPTIONS ET EQUIPEMENTS :Extérieur :- rétroviseurs électriques- jantes alliageIntérieur et confort :- ordinateur de bord- bluetooth- GPSSécurité :- détecteur de pluie- airbags frontaux- airbags latéraux- radar arrière de détection d'obstacles- régulateur de vitesseAutres équipements et informations :kilométrage garanti, puissance réelle : 90 ch, radar d'aide au stationnement, vitres avant éléctri

Brie-Comte-Robert 77170
                
Marque
Renault
Modèle
Zoe
Année-modèle

                    
                        2017
                            
                        
                    
                
Kilométrage
1 KM
Carburant
Electrique
Boîte de vitesse
Automatique
Référence
vo221054
Description :
Renault Zoe Intens Gamme 2017 berline, gris titanium, 1 cv, 5 portes, mise en circulation le 27/09/2017, garantie 24 mois.OPTIONS ET EQUIPEMENTS :Sécurité :- radar arrière de détection d'obstaclesAutres équipements et informations :kilométrage garanti, puissance réelle : 90 ch, radar d'aide au stationnementRéférence annonce : VO221054NOUS CONTACTER :0160625056

                            	Renault Zoe Intens Gamme 2017
                                
                            	
							
(pro)

                                            17 790 €
                                            
                                        
Prix

                    
          

Prix

                    
                            11 990 €
                        
                    
                    
                

Ville


Pantin 93500
                
Marque
Renault
Modèle
Zoe
Année-modèle

                    
                        2015
                            
                        
                    
                
Kilométrage
19 756 KM
Carburant
Electrique
Boîte de vitesse
Manuelle
Référence
p178001391
Description :
Renault Zoe INTENS berline, noir, 1 cv, 5 portes, mise en circulation le 05/01/2015, garantie 12 mois.OPTIONS ET EQUIPEMENTS :Extérieur :- rétroviseurs électriques- caméra de recul- jantes aluminiumIntérieur et confort :- climatisation automatiqueSécurité :- rétroviseurs dégivrants- radar d'aide au stationnement- régulateur de vitesse- limiteur de vitesseAutres équipements et informations :vitres avant éléctriques, Vitres arrières électriquesRéférence annonce : P178001391NOUS CONTACTER :0174730257

                       

[]